In [1]:
import sys
import os
import src.data_processing as dp
from src.utils import *
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
import torch
from model.losses import EVL
from model.HailNet import HailNet

In [8]:
target_adygeya, data = dp.get_target((1991, 2021), region = 'Республика Адыгея', freq="Daily")
target_kras, data = dp.get_target((1991, 2021), region = 'Краснодарский край', freq="Daily")

In [14]:
subjects = data[data["Название явления "] == "Град"][["Субъект Российской Федерации "]].groupby(by = "Субъект Российской Федерации ").count()

In [4]:
print(data[data["Название явления "] == "Град"][["Субъект Российской Федерации "]].value_counts())

Субъект Российской Федерации       
Краснодарский край                     93
Ставропольский край                    87
Республика Северная Осетия - Алания    48
Карачаево-Черкесская Республика        32
Кабардино-Балкарская Республика        32
                                       ..
Брянская область                        1
Удмуртcкая Республика                   1
Владимирская область                    1
Республика Карелия                      1
Ярославская область                     1
Length: 62, dtype: int64


In [11]:
target_kras[target_kras["target"] == 1].index

DatetimeIndex(['1992-06-27', '1992-07-17', '1994-04-21', '1994-06-09',
               '1999-08-03', '2000-09-07', '2001-06-21', '2002-05-30',
               '2004-06-01', '2004-06-07', '2005-06-01', '2005-08-18',
               '2006-06-08', '2006-06-16', '2007-05-25', '2007-08-08',
               '2007-08-16', '2007-09-08', '2008-04-28', '2008-07-02',
               '2008-07-06', '2008-07-26', '2008-08-19', '2009-06-04',
               '2009-06-12', '2009-06-22', '2009-06-27', '2009-06-30',
               '2009-09-05', '2010-05-15', '2010-05-17', '2010-05-18',
               '2010-05-23', '2010-06-05', '2010-06-23', '2011-06-26',
               '2011-07-04', '2011-08-02', '2011-08-16', '2011-09-01',
               '2012-05-17', '2012-05-21', '2012-05-31', '2012-06-16',
               '2012-07-19', '2013-05-14', '2013-05-25', '2013-06-01',
               '2013-06-07', '2013-06-15', '2013-07-01', '2013-09-20',
               '2014-05-18', '2014-05-19', '2014-06-09', '2014-06-11',
      

In [9]:
target_adygeya[target_adygeya["target"] == 1]

,target
2000-09-07,1
2002-05-30,1
2004-06-08,1
2006-05-09,1
2006-06-08,1
2007-06-19,1
2008-04-17,1
2009-06-13,1
2009-07-13,1
2011-07-04,1


In [39]:
a - 1

array([ 8,  2,  6, 20, 29,  0,  6,  0, 17,  7, 15, 24,  7, 15,  7, 27,  1,
        5, 25, 18,  3, 11, 21, 26, 29,  4, 14, 16, 17, 22,  4, 22, 25,  3,
        1, 15,  0, 16, 20, 30, 15, 18, 13, 24,  0,  6, 14,  0, 19, 17, 18,
        8, 10, 19, 28, 29,  0, 22, 17, 18, 25, 14, 22, 29, 18, 12, 24, 25,
       26,  3,  5, 28, 29,  0, 24, 29,  1, 10, 22,  5, 14, 16, 12, 27,  5,
        0, 15, 24, 25,  0])

In [43]:
aq = np.array([  9,  3,  7, 21, 30,  1,  7,  1, 18,  8, 16, 25,  8, 16,
             8, 28,  2,  6, 26, 19,  4, 12, 22, 27, 30,  5, 15, 17, 18, 23,  5,
            23, 26,  4,  2, 16,  1, 17, 21, 31, 16, 19, 14, 25,  1,  7, 15,  1,
            20, 18, 19,  9, 11, 20, 29, 30,  1, 23, 18, 19, 26, 15, 23, 30, 19,
            13, 25, 26, 27,  4,  6, 29, 30,  1, 25, 30,  2, 11, 23,  6, 15, 17,
            13, 28,  6,  1, 16, 25, 26,  1])


a = np.append(aq - 1, aq + 1)

In [49]:
a

array([ 8,  2,  6, 20, 29,  0,  6,  0, 17,  7, 15, 24,  7, 15,  7, 27,  1,
        5, 25, 18,  3, 11, 21, 26, 29,  4, 14, 16, 17, 22,  4, 22, 25,  3,
        1, 15,  0, 16, 20, 30, 15, 18, 13, 24,  0,  6, 14,  0, 19, 17, 18,
        8, 10, 19, 28, 29,  0, 22, 17, 18, 25, 14, 22, 29, 18, 12, 24, 25,
       26,  3,  5, 28, 29,  0, 24, 29,  1, 10, 22,  5, 14, 16, 12, 27,  5,
        0, 15, 24, 25,  0, 10,  4,  8, 22, 31,  2,  8,  2, 19,  9, 17, 26,
        9, 17,  9, 29,  3,  7, 27, 20,  5, 13, 23, 28, 31,  6, 16, 18, 19,
       24,  6, 24, 27,  5,  3, 17,  2, 18, 22, 32, 17, 20, 15, 26,  2,  8,
       16,  2, 21, 19, 20, 10, 12, 21, 30, 31,  2, 24, 19, 20, 27, 16, 24,
       31, 20, 14, 26, 27, 28,  5,  7, 30, 31,  2, 26, 31,  3, 12, 24,  7,
       16, 18, 14, 29,  7,  2, 17, 26, 27,  2])

In [11]:
feature_names = ["mean_2m_air_temperature"]
x, y = dp.get_traindl((2001, 2002), feature_names, "data/Tambov", sequence_length = 12,  freq="Monthly")

Hello, motherfucker!!!


In [10]:
for (xb, yb) in x:
    print(xb.shape)
    break

In [12]:
y.shape

torch.Size([12, 12, 1, 234, 346])

In [4]:
import os
import glob

dir_path = "data/Krasnodarskiy/No Hail/"
paths = glob.glob(dir_path + "*")
for p in paths:
    if len(os.listdir(p)) != 9:
        print(p)
        #raise ValueError("alarm")
print("Alright!")

Alright!


In [14]:
no_hail_dir_path = "data/Krasnodarskiy/No Hail/"
hail_dir_path = "data/Krasnodarskiy/Hail/"
hail_paths = glob.glob(hail_dir_path + "*")
no_hail_paths = glob.glob(no_hail_dir_path + "*")
intersection = set(hail_paths).intersection(no_hail_paths) 
print(intersection)

set()


In [15]:
hail_dates = [p[-9:] for p in hail_paths]
no_hail_dates = [p[-9:] for p in no_hail_paths]
intersection = set(hail_dates).intersection(no_hail_dates) 
print(intersection)

set()


In [2]:
tdl, x = dp.get_traindl(feature_names = [
  'dewpoint_temperature_2m',
  'runoff', 
  'snowmelt',
  'snowfall', 
  'surface_latent_heat_flux',
  'potential_evaporation',
  'soil_temperature_level_1',
  'surface_net_solar_radiation', 
  'surface_solar_radiation_downwards'
  ])

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4959714240 bytes.

In [4]:
x[:, 0].shape

torch.Size([15, 5, 359, 533])

In [4]:
len(y)

93

In [13]:
x[0].shape

TypeError: 'DataLoader' object is not subscriptable

In [17]:
x = torch.tensor([[1, 2], [3,4]])
y = torch.tensor([[2, 3], [4, 4]])
a = torch.cat([x, y], dim = 1)
a

tensor([[1, 2, 2, 3],
        [3, 4, 4, 4]])

In [8]:
a = a.reshape(3, 2)

In [13]:
a.reshape(2, -1)

tensor([[1, 2, 3],
        [2, 3, 4]])

In [ ]:
a = 